In [1]:
import pandas as pd
import numpy as np
import sqlalchemy
from urllib.parse import urlencode
import requests
from requests import Response
import datetime as dt
from datetime import time, date, timedelta
from dateutil.relativedelta import relativedelta
import itertools
from math import ceil
import scipy.stats as stats

In [9]:
from ta.trend import CCIIndicator
def CCI(window_para, highband, to_csv):
    lowband = -highband
    result = pd.read_csv('raw-weekly-pricevolume.csv')
    codes = result['code'].unique().tolist()
    for code in codes:
        close_weekly = result[result['code']==code]['close_weekly']
        high_weekly = result[result['code']==code]['high_weekly']
        low_weekly = result[result['code']==code]['low_weekly']
        result.loc[code == result['code'].shift(window_para),'cci'] = CCIIndicator(high = high_weekly, low = low_weekly, close=close_weekly, window=window_para, constant = 0.02).cci()
    result.loc[result['code'] == result['code'].shift(1),'return'] = result['close_weekly']/result['close_weekly'].shift(1) - 1
    result = result.dropna()
    result.loc[(result['code'] == result['code'].shift(1))&(result['cci']>highband),'signal'] = 1
    result.loc[(result['code'] == result['code'].shift(1))&(result['cci']<lowband),'signal'] = 0
    result.loc[result['code'] != result['code'].shift(1),'signal'] = 0
    result = result.ffill()
    
    result.loc[result['code'] == result['code'].shift(1),'daily_return'] = result['signal'].shift(1)*result['return']
    result.loc[(result['code'] == result['code'].shift(1))&(result['signal']==1)&(result['signal'].shift(1)==0),'daily_return']=result['daily_return']-0.005
    
    dailyprofit = pd.pivot(result, index = 'date', columns = 'code', values = 'daily_return')
    dailyprofit = dailyprofit.replace(0, np.nan)
    dailyprofit.reset_index(inplace = True)
    day_return = dailyprofit.copy()
    day_return['count'] = day_return.iloc[:,1:len(dailyprofit.columns.to_list())].count(axis = 1) #exclude date column
    day_return.loc[day_return['count']<5,'profit'] = day_return.iloc[:,1:len(dailyprofit.columns.to_list())].mean(axis = 1,skipna=True)*(day_return['count']/5)
    day_return.loc[day_return['count']==0,'profit'] = 0
    day_return.loc[day_return['count']>=5,'profit'] = day_return.iloc[:,1:len(dailyprofit.columns.to_list())].mean(axis = 1,skipna=True)
    
    profit = (day_return['profit'] +1).cumprod()
    day_return['cumulative'] = profit
    if to_csv:
        day_return.to_csv('checkrsi.csv')
    cul_arr = day_return['cumulative'].to_list()
    dd_arr = []
    maxcul = 0
    for x in day_return.cumulative.dropna().tolist():
        if x >= maxcul:
            maxcul = x
            dd_arr.append(0)
        else:
            dd_arr.append((maxcul-x)/maxcul)
    print(f'MDD {max(dd_arr)}')
    return float(profit[-1:]), window_para, highband

In [10]:
optimize_result = []
for window_para in range(2,30, 2):
    for highband in range(50,200, 5):
        x=CCI(window_para,highband, to_csv = False)
        optimize_result.append(x)    
optimize_result = pd.DataFrame(optimize_result)

MDD 0.2097142844642606
MDD 0.21066620829840357
MDD 0.21066620829840357
MDD 0.21066620829840357
MDD 0.21066620829840357
MDD 0.21066620829840357
MDD 0.21066620829840357
MDD 0.21066620829840357
MDD 0.21066620829840357
MDD 0.21066620829840357
MDD 0.2528664776570716
MDD 0.2528664776570716
MDD 0.2528664776570716
MDD 0.2528664776570716
MDD 0.2528664776570716
MDD 0.2528664776570716
MDD 0.2528664776570716
MDD 0.2528664776570716
MDD 0.2528664776570716
MDD 0.2528664776570716
MDD 0.2528664776570716
MDD 0.2528664776570716
MDD 0.2528664776570716
MDD 0.2528664776570716
MDD 0.2528664776570716
MDD 0.2528664776570716
MDD 0.2528664776570716
MDD 0.2528664776570716
MDD 0.2528664776570716
MDD 0.2528664776570716
MDD 0.31355743798794344
MDD 0.2952376590488743
MDD 0.2984620497941066
MDD 0.3013862533113992
MDD 0.28934165805695367
MDD 0.28409948285797976
MDD 0.28449577933644404
MDD 0.26593391133538846
MDD 0.26121833546555767
MDD 0.23264255183358742
MDD 0.18356219122100317
MDD 0.2673569590117649
MDD 0.26735695901

MDD 0.2096452482719421
MDD 0.21744874487192925
MDD 0.21887207488683302
MDD 0.2129077690584678
MDD 0.21037611706068743
MDD 0.21384667840036942
MDD 0.22793951107176136
MDD 0.21735142640913044
MDD 0.20191974245868563
MDD 0.20228775643232697
MDD 0.21018132796901765
MDD 0.21217563300785044
MDD 0.21703120493631478
MDD 0.22169217627895726
MDD 0.21764886970847425
MDD 0.22410095368570326
MDD 0.2218371065199563
MDD 0.213242570668946
MDD 0.2162831847990018
MDD 0.2086502530155682
MDD 0.208516231679367
MDD 0.20605407208720017
MDD 0.21412258021540034
MDD 0.2111862390929902
MDD 0.20667545640763682
MDD 0.21686530868258325
MDD 0.20973671493920573
MDD 0.21248298789610243
MDD 0.21573409050670758
MDD 0.22207392753125374
MDD 0.22126513863610794
MDD 0.22215896134208876
MDD 0.22187587233175302
MDD 0.22244299067933268
MDD 0.2244912043575433
MDD 0.2181516406769347
MDD 0.21144629909318363
MDD 0.20490309015289257
MDD 0.1999469685496838
MDD 0.20427412369588932
MDD 0.20855022952384078
MDD 0.20998088542586102
MDD 0

In [5]:
CCI(4,140, to_csv = False)

MDD 0.2673569590117649


(4.789042619936213, 4, 140)

In [8]:
optimize_result.sort_values(by=0, ascending =False ).head(50)

,0,1,2
23,3.814153,2,165
20,3.814153,2,150
17,3.814153,2,135
16,3.814153,2,130
15,3.814153,2,125
14,3.814153,2,120
13,3.814153,2,115
12,3.814153,2,110
11,3.814153,2,105
21,3.814153,2,155
